Code to call the OpenAI API to call and solve visual puzzles (ConceptARC problems).

https://github.com/victorvikram/ConceptARC/tree/main/MinimalTasks


In [37]:
from openai import OpenAI
import dotenv
import os
from rich import print as rprint # for making fancy outputs

dotenv.load_dotenv()

client = OpenAI()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Call ConceptARC solver. Start by creating a good system prompt.

In [38]:
system_prompt = "You are an intelligent solver of puzzles."
user_prompt = 'your job is to solve a puzzle. These are puzzles in json format. I have some training examples and a test example in ConceptARC. Please solve it. Here is the puzzle:  {"train":[{"input":[[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,4,0,0,0,0,0],[0,0,0,4,4,4,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,3,3,3,0,0,0,0],[0,0,0,3,3,3,0,0,0,0],[0,0,0,3,3,3,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0]],"output":[[0,4,0],[4,4,4]]},{"input":[[0,0,3,3,0,0,0,0],[0,0,3,3,0,0,0,0],[0,0,0,0,0,0,0,0],[0,4,0,4,0,0,0,0],[0,4,4,4,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]],"output":[[3,3],[3,3]]},{"input":[[0,0,0,0,0,0,0,0,0],[0,0,0,3,3,3,3,3,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,4,4,4,4,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]],"output":[[3,3,3,3,3]]},{"input":[[0,0,0,0,0,0,0,0],[0,0,0,4,0,0,0,0],[0,0,4,0,4,0,0,0],[0,0,0,4,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,3,3,3,3,0,0],[0,0,3,0,0,3,0,0],[0,0,0,3,3,0,0,0]],"output":[[0,4,0],[4,0,4],[0,4,0]]}],"test":[{"input":[[4,4,4,0,0,0],[4,4,4,0,0,0],[0,0,0,0,0,0],[0,0,3,0,0,0],[3,3,3,3,0,0],[0,0,0,0,0,0]],"output":[[4,4,4],[4,4,4]]},{"input":[[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[4,4,4,4,4,4,4,4,4,4],[3,3,3,3,3,3,3,3,3,3],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0]],"output":[[4,4,4,4,4,4,4,4,4,4]]},{"input":[[0,0,0,0,0],[0,3,3,3,0],[0,3,0,3,0],[0,3,3,3,0],[0,4,0,4,0],[4,0,4,0,4]],"output":[[3,3,3],[3,0,3],[3,3,3]]}]}'


Solve the problem now.

In [39]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content": system_prompt},
        {"role": "user",  "content": user_prompt}
    ],
    temperature=1.0,
    max_tokens=1000,
)

print(response.choices[0].message.content) # print the response from the model


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************CmkA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

Use a vision model now to solve ConceptARC puzzle


In [ ]:
prompt = ("You are an intelligent solver of puzzles. " 
"Your job is to solve a puzzle. The puzzle is given to you as an image " 
"and you need to provide the solution in text format. "
"Please provide the solution in a clear and concise manner. "
"Make sure to include all necessary details in the solution. "
"Feel free to make any inferences you need to. "
)





Now call the OpenAI vision model API

In [ ]:
import base64
import requests
import json

function to encode the image

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string



In [ ]:
# Path to image file
image_path = "imgs/puzzle.png"

def get_image_caption(image_path, prompt):
  # Getting the base64 string
  base64_image = encode_image(image_path)

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {OPENAI_API_KEY}"
  }

  payload = {
    "model": "gpt-4o-mini",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 512
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

  return response.json()['choices'][0]['message']['content']




Pass this to the solver

In [ ]:
caption = get_image_caption(image_path, prompt)

#print("Caption for the image:")
#print(caption)
rprint(caption) # print the caption

The puzzle consists of a series of grids with colored shapes and corresponding yellow segments on the right side. 
The goal is to identify relationships between the shapes to determine the next correct segment.

Observations of the patterns in each grid:

1. **First Grid**: Shapes in various colors.
2. **Second Grid**: Similar color scheme with some shapes missing. 
3. **Third Grid**: A combination of shapes has been removed before adding yellow shapes.
4. **Fourth Grid**: Shift in shape colors with orange being prominent.
5. **Fifth Grid**: Balanced shapes with new color entries.

The yellow segments indicate completion or necessity in the grid. It seems that based on the previous patterns, the
sequence follows a logic of retained colors with a certain shape incorporation.

**Solution**: The segment that is missing in each grid corresponds to what needs to be filled in the next. Based on
this observation and patterns deduced, the final correct segment to follow the last grid would be:

- **Fill the yellow segment to complete the last shape pattern on the right side.**

In summary, the answer is to fill in the next yellow segment as indicated in the derived pattern from the grids 
preceding.

Solve a puzzle from H. Dudeney's classic book "Canterbury Puzzles"

In [ ]:
prompt = ("You are an intelligent solver of puzzles. " 
"Your job is to solve a puzzle. The puzzle is given to you as an image " 
"and you need to provide the solution in text format. "
"Please provide the solution in a clear and concise manner. "
"Make sure to include all necessary details in the solution. "
"Feel free to make any inferences you need to. "
"Please check your answer again and recheck your solutions. "
)

Solve the puzzles

In [ ]:
#  Path to image file
image_path = "imgs/squirepuzzle.jpg"

caption = get_image_caption(image_path, prompt)

rprint(caption) # print the caption

To solve the puzzle presented by Master Herbert Spearing, we need to find two cubes of silver that together contain
exactly 17 cubic inches.

We know:
- Each cube's volume, \( V = s^3 \), where \( s \) is the side length of the cube.
- The total volume should be \( V_1 + V_2 = 17 \).

Let the side lengths of the two cubes be \( a \) and \( b \). Then:
[ 
a^3 + b^3 = 17 
\]

To find integers \( a \) and \( b \):
1. The possible values for \( a \) (where \( a^3 < 17 \)) are 1, 2, and 3 since \( 3^3 = 27 \) exceeds 17.
2. We evaluate combinations:

   - \( a = 1 \): 
     [ 
     1^3 + b^3 = 17 \implies b^3 = 16 \implies b = 2.52 \quad (\text{not an integer}) 
     \]

   - \( a = 2 \): 
     [ 
     2^3 + b^3 = 17 \implies 8 + b^3 = 17 \implies b^3 = 9 \implies b = 2.08 \quad (\text{not an integer}) 
     \]

   - \( a = 3 \): 
     [ 
     3^3 + b^3 = 17 \implies 27 + b^3 = 17 \quad (\text{not valid since } b^3 < 0) 
     \]

After checking every integer possibility and realizing the cubes must fit perfectly without remainders or 
fractions, we find a combination. Thus, we can determine that:

A solution is:
- Cube 1: side length \( a = 1 \) inch (Volume = \( 1^3 = 1 \) cubic inch)
- Cube 2: side length \( b = 2 \) inches (Volume = \( 2^3 = 8 \) cubic inches)
- Total volume = \( 1^3 + 2^3 = 1 + 8 = 9 \), which does not work.

Thus, the dimensions that work would be where \( a = 1 \) and \( b = 3 \):
-

Code to solve ARC-AGI 2 problem

https://arcprize.org/blog/announcing-arc-agi-2-and-arc-prize-2025

In [ ]:
prompt = ("You are an intelligent solver of puzzles. " 
"Your job is to solve a puzzle (ARC-AGI 2). This is the Abstraction and Reasoning Corpus 2. The puzzle is given to you as an image " 
"and you need to provide the solution in text format. "
"Please provide the solution in a clear and concise manner. "
"Make sure to include all necessary details in the solution. "
"Feel free to make any inferences you need to. "
)

In [ ]:
#  Path to image file
image_path = "imgs/arcagi2.png"

caption = get_image_caption(image_path, prompt)

rprint(caption) # print the caption

KeyError: 'choices'